In [ ]:
!pip3 install datasets seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 8.1 MB/s 
     |████████████████████████████████| 43 kB 2.3 MB/s 
     |████████████████████████████████| 132 kB 71.0 MB/s 
     |████████████████████████████████| 182 kB 78.7 MB/s 
     |████████████████████████████████| 212 kB 67.5 MB/s 
     |████████████████████████████████| 127 kB 74.2 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16179 sha256=50c31fc4c9430b53cbd9410e02891667338fb00efcfab3e8c53e1279c029d06e
  Stored in directory: /root/.cache/pip/wheels/ad/5c/ba/05fa33fa5855777b7d686e843ec07452f22a66a138e290e732
Successfully built seqeval
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
!pip3 install git+https://github.com/huggingface/transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-hieiexjq
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-hieiexjq
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 7.6 MB 9.1 MB/s 
  Created wheel for transformers: filename=transformers-4.26.0.dev0-py3-none-any.whl size=5964818 sha256=19eae6a37a9b4300597296a53cd6123daff63b7c4a7e614e2b96cc109f8e46c9
  Stored in directory: /tmp/pip-ephem-wheel-cache-ii4_ki7p/wheels/42/68/45/c63edff61c292f2dfd4df4ef6522dcbecc603e7af82813c1d7
Successfully built transformers


In [ ]:
import datasets
dataset = datasets.load_dataset("nvm472001/cvdataset-layoutlmv3")

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


/root/.cache/huggingface/datasets/downloads/extracted/add636b866139f17211d62f7d1a2d6c68d6372025c38a9918437340ec5001d04/data1/raw/img/imgCV_208.jpg
/root/.cache/huggingface/datasets/downloads/extracted/add636b866139f17211d62f7d1a2d6c68d6372025c38a9918437340ec5001d04/data1/raw/img/imgCV_904.jpg
/root/.cache/huggingface/datasets/downloads/extracted/add636b866139f17211d62f7d1a2d6c68d6372025c38a9918437340ec5001d04/data1/raw/img/imgCV_630.jpg


Generating test split: 0 examples [00:00, ? examples/s]

Dataset cvdataset-layoutlmv3 downloaded and prepared to /root/.cache/huggingface/datasets/nvm472001___cvdataset-layoutlmv3/CV Extractions/1.0.0/1414f5a66b931c9f9592e64a3a9e424e7f5b2195318a5ff2fccb2e3446fd5274. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from transformers import AutoProcessor
processor = AutoProcessor.from_pretrained("microsoft/layoutlmv3-large", apply_ocr=False)

Downloading:   0%|          | 0.00/275 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/857 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [ ]:
from datasets.features import ClassLabel

features = dataset["train"].features
column_names = dataset["train"].column_names
image_column_name = "image_path"
text_column_name = "words"
boxes_column_name= "bboxes"
label_column_name = "ner_tags"

def get_label_list(labels):
  unique_labels = set()

  for label in labels:
    unique_labels = unique_labels | set(label)
  
  labels_list = list(unique_labels)
  labels_list.sort()

  return labels_list

if isinstance(features[label_column_name].feature, ClassLabel):
  label_list = features[label_column_name].feature.names  
else:
  label_list = get_label_list(dataset["train"][label_column_name])
id2label = {k:v for k, v in enumerate(label_list)}
label2id = {v:k for k, v in enumerate(label_list)}
num_label = len(id2label)

In [ ]:
from PIL import Image

def prepare_examples(examples):
  images = [Image.open(path).convert("RGB") for path in examples['image_path']] #Image.open(examples[image_column_name])
  words = examples[text_column_name]
  boxes = examples[boxes_column_name]
  word_labels = examples[label_column_name]

  encoding = processor(images, words, boxes=boxes, word_labels=word_labels,
                       truncation=True, padding="max_length")

  return encoding

In [ ]:
from datasets import Features, Sequence, ClassLabel, Value, Array2D, Array3D

# we need to define custom features for `set_format` (used later on) to work properly
features = Features({
    'pixel_values': Array3D(dtype="float32", shape=(3, 224, 224)),
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'attention_mask': Sequence(Value(dtype='int64')),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),
    'labels': Sequence(feature=Value(dtype='int64')),
})

In [ ]:
train_dataset = dataset["train"].map(
    prepare_examples,
    batched=True,
    batch_size=5,
    remove_columns=column_names,
    features=features,
)

  0%|          | 0/134 [00:00<?, ?ba/s]

In [ ]:
eval_dataset = dataset["test"].map(
    prepare_examples,
    batched=True,
    batch_size=5,
    remove_columns=column_names,
    features=features,
)

  0%|          | 0/34 [00:00<?, ?ba/s]

In [ ]:
train_dataset.set_format('torch')
eval_dataset.set_format('torch')

In [ ]:
from datasets import load_metric

metric = load_metric("seqeval")

<ipython-input-11-edef154daec7>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [ ]:
import numpy as np

return_entity_level_metrics = False

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    if return_entity_level_metrics:
        # Unpack nested dictionaries
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

In [ ]:
from transformers import LayoutLMv3ForTokenClassification

model = LayoutLMv3ForTokenClassification.from_pretrained("microsoft/layoutlmv3-large",
                                                         id2label=id2label,
                                                         label2id=label2id)

Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-large and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
                    output_dir="/content/drive/MyDrive/weights/layoutlmv3-fine-tuning-cvdataset/large",
                    max_steps=5000,
                    per_device_train_batch_size=2,
                    per_device_eval_batch_size=2,
                    learning_rate=1e-5,
                    eval_steps=500,
                    evaluation_strategy="steps",
                    metric_for_best_model="f1"
                )

In [ ]:
from transformers.data.data_collator import default_data_collator

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor,
    data_collator=default_data_collator,
    compute_metrics=compute_metrics,
)

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
import torch

torch.cuda.empty_cache()

In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 666
  Num Epochs = 16
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 5000
  Number of trainable parameters = 357075476
/usr/local/lib/python3.8/dist-packages/transformers/modeling_utils.py:801: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
500,0.543700,0.234174,0.831942,0.768243,0.798824,0.934286
1000,0.194500,0.230753,0.870840,0.793422,0.830331,0.947588
1500,0.120200,0.247093,0.867964,0.847893,0.857811,0.953147
2000,0.092800,0.261853,0.883124,0.830935,0.856235,0.954338
2500,0.066500,0.284951,0.881657,0.842240,0.861498,0.953742
3000,0.047900,0.293342,0.885201,0.836074,0.859937,0.954536
3500,0.031800,0.304235,0.875534,0.842240,0.858565,0.951757
4000,0.027000,0.317899,0.895709,0.847379,0.870874,0.956125
4500,0.014700,0.341094,0.886389,0.829908,0.857219,0.952750
5000,0.013100,0.324432,0.881429,0.836588,0.858423,0.953941


***** Running Evaluation *****
  Num examples = 170
  Batch size = 2
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: undefined seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: experience_name seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: experience_time seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: person_name seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: information seems not to be NE tag.
  warnings.warn('{} seems not to be

TrainOutput(global_step=5000, training_loss=0.11522001304626465, metrics={'train_runtime': 7467.6444, 'train_samples_per_second': 1.339, 'train_steps_per_second': 0.67, 'total_flos': 9350504079360000.0, 'train_loss': 0.11522001304626465, 'epoch': 15.02})

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 170
  Batch size = 2
/usr/local/lib/python3.8/dist-packages/transformers/modeling_utils.py:801: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: undefined seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: experience_name seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: experience_time seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: person_name seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: information seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.8/dist-packages/seqev

{'eval_loss': 0.3244318962097168,
 'eval_precision': 0.881429344883595,
 'eval_recall': 0.8365878725590956,
 'eval_f1': 0.8584234115475876,
 'eval_accuracy': 0.9539408378002779,
 'eval_runtime': 40.8236,
 'eval_samples_per_second': 4.164,
 'eval_steps_per_second': 2.082,
 'epoch': 15.02}

In [ ]:
# from transformers import AutoModelForTokenClassification

# model = AutoModelForTokenClassification.from_pretrained("/content/drive/MyDrive/weights/layoutlmv3-fine-tuning-cvdataset/checkpoint-2000")

In [ ]:
# from transformers import LayoutLMv3Processor

# processor = LayoutLMv3Processor.from_pretrained("/content/drive/MyDrive/weights/layoutlmv3-fine-tuning-cvdataset/checkpoint-2000")

In [ ]:
# example = dataset["test"][0]
# print(example.keys())

In [ ]:
# example["bboxes"]

In [ ]:
# from PIL import Image
# image = Image.open(example["image_path"]).convert("RGB")
# words = example["words"]
# boxes = example["bboxes"]
# word_labels = example["ner_tags"]

# encoding = processor(image, words, boxes=boxes, return_tensors="pt", truncation=True)
# for k,v in encoding.items():
#   print(k,v.shape)

In [ ]:
# import torch
# with torch.no_grad():
#   outputs = model(**encoding)

In [ ]:
# logits = outputs.logits
# logits.shape

In [ ]:
# predictions = logits.argmax(-1).squeeze().tolist()
# print(predictions)

In [ ]:
# def unnormalize_box(bbox, width, height):
#      return [
#          width * (bbox[0] / 1000),
#          height * (bbox[1] / 1000),
#          width * (bbox[2] / 1000),
#          height * (bbox[3] / 1000),
#      ]

# token_boxes = encoding.bbox.squeeze().tolist()
# width, height = image.size

# true_predictions = [model.config.id2label[pred] for pred, label in zip(predictions, labels) if label != - 100]
# true_labels = [model.config.id2label[label] for prediction, label in zip(predictions, labels) if label != -100]
# true_boxes = [unnormalize_box(box, width, height) for box, label in zip(token_boxes, labels) if label != -100]

In [ ]:
# from PIL import ImageDraw, ImageFont

# draw = ImageDraw.Draw(image)
# font = ImageFont.load_default()

In [ ]:
# import random

# def generate_label2color(label_list):
#   num_of_colors = len(label_list)
#   color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
#              for i in range(num_of_colors)]
#   label2color = {k:v for k, v in zip(label_list, color)}
#   return label2color

# label2color = generate_label2color(label_list)

In [ ]:
for prediction, box in zip(true_predictions, true_boxes):
  draw.rectangle(box, outline=label2color[prediction])
  draw.text((box[0] + 10, box[1] - 10), text=prediction, fill=label2color[prediction])

image

NameError: ignored

In [ ]:
!zip -r /content/ /content/drive/MyDrive/weights/layoutlmv3-fine-tuning-cvdataset/large/

In [ ]:
%cd /content/drive/MyDrive/weights/layoutlmv3-fine-tuning-cvdataset/large/

/content/drive/MyDrive/weights/layoutlmv3-fine-tuning-cvdataset/large


In [ ]:
%ls

checkpoint-1000/  checkpoint-2500/  checkpoint-4000/  checkpoint-5000/
checkpoint-1500/  checkpoint-3000/  checkpoint-4500/  checkpoint.zip
checkpoint-2000/  checkpoint-3500/  checkpoint-500/   runs/


In [ ]:
!zip -r checkpoint ./checkpoint-5000

  adding: checkpoint-5000/ (stored 0%)
  adding: checkpoint-5000/config.json (deflated 64%)
  adding: checkpoint-5000/pytorch_model.bin (deflated 7%)
  adding: checkpoint-5000/preprocessor_config.json (deflated 51%)
  adding: checkpoint-5000/tokenizer_config.json (deflated 78%)
  adding: checkpoint-5000/special_tokens_map.json (deflated 85%)
  adding: checkpoint-5000/vocab.json (deflated 59%)
  adding: checkpoint-5000/merges.txt (deflated 53%)
  adding: checkpoint-5000/tokenizer.json (deflated 72%)
  adding: checkpoint-5000/training_args.bin (deflated 49%)
  adding: checkpoint-5000/optimizer.pt (deflated 19%)
  adding: checkpoint-5000/scheduler.pt (deflated 50%)
  adding: checkpoint-5000/trainer_state.json (deflated 76%)
  adding: checkpoint-5000/rng_state.pth (deflated 28%)
